# Cifar10 with tf.keras, tf.data and image augmentation

In [ ]:
import sys
import shutil
import tensorflow as tf

import numpy as np
%matplotlib inline
import random
import matplotlib.pyplot as plt

In [ ]:
# Python version 3.5 or 3.6
assert sys.version_info >= (3, 5)
assert sys.version_info < (3, 7)
# Tensorflow 2.0
assert tf.__version__ >= "2.0"

The problem we are trying to solve here is to classify RGB images (32 pixels by 32 pixels), into their 10 categories (_airplane_, _automobile_, _bird_, _cat_, _deer_, _dog_, _frog_, _horse_, _ship_, _truck_). The dataset we will use is the CIFAR10 dataset, a classic dataset in the machine learning community.

# Input Data Management

## Download the dataset

The CIFAR10 dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays.

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

## Visualize the data

Let's print the shapes of the datasets

In [ ]:
print("Train images shape : {}".format(train_images.shape))
print("Train labels shape : {}".format(train_labels.shape))
print("Test images shape : {}".format(test_images.shape))
print("Test labels shape : {}".format(test_labels.shape))

Let's see how the images look. This function shows a random example along with it's corresponding label.

In [ ]:
i = random.randint(0, 100)

print("Label: %s" % train_labels[i])
plt.imshow(train_images[i], cmap='gray')

Thats a little blurry !

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we will verify if these predictions match the labels from `test_labels`.

## Create a tf.data Dataset

We will implement the input preprocessing function, composed of the following steps :
- `cast` image to float32
- `divide` image values by 255
- `one_hot` encoding of the label

In [ ]:
NUM_CAT = 10

def _preprocess_inputs(img, label):
    # Cast image to float32
    img_float = tf.cast(img, tf.float32)
    # Divide image values by 255
    img_norm = tf.divide(img_float, 255)
    
    label_int = label[0]
    # One Hot encoding of the label
    label_one_hot = tf.one_hot(label_int, depth=NUM_CAT)
    
    return img_norm, label_one_hot

We will apply image augmentation in the input processing pipeline. Here are the transformations we can apply :
- `random_flip_left_right` : Flip the image
- `random_brightness` : Randomly modify image brightness (use the `max_delta`, for example with a value of 32.0/255)
- `random_saturation` : Randomly modify image brightness (use the `lower` and `upper` arguments, for example with values 0.5 and 1.5 respectively)

Finally, in order to make sure the image values are still in the [0,1] interval, use the `clip_by_value` function.

Remember to only apply these transformations to the train set, not the test set, as we want the test images to be the real ones.

In [ ]:
def _image_augmentation(image, label):
    # Flip image
    image = tf.image.random_flip_left_right(image)
    # Random Brightness
    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    # Random Saturation
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    # Clip values between 0 and 1
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

We will now use `tf.data` to create the input data pipeline. It will be composed of the following steps :
- Create the Dataset. Since we already have in-memory data, we will use the `from_tensor_slices` function
- `map` the dataset with the preprocessing function implemented above
- `map` the result with the image augmentation function
- Add the `shuffle`, `repeat`, `batch` and `prefect` steps to configure training

In [ ]:
BATCH_SIZE = 32
SHUFFLE_SIZE = 10000
NUM_EPOCHS = 5

# Create Dataset
ds_train = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
# Map for input preprocessing
ds_train = ds_train.map(_preprocess_inputs)
# Map for image augmentation
ds_train = ds_train.map(_image_augmentation)
# Shuffle / Repeat / Batch / Prefetch
ds_train = ds_train.shuffle(SHUFFLE_SIZE).repeat(NUM_EPOCHS).batch(BATCH_SIZE).prefetch(1)

The input data pipeline for the test set is quite similar, except there is no need to use the `repeat` and `prefetch` steps. Moreover, the image augmentation phase is not necessary for the test set, as we want the real images to be used to evaluate the results.

In [ ]:
# Create Dataset
ds_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
# Map
ds_test = ds_test.map(_preprocess_inputs)
# Shuffle / Repeat / Batch / Prefetch
ds_test = ds_test.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)

# Model Management

## Build the model

Our Neural Network will now be composed of a convolutional based coming from a trained network called `NASNetMobile`, followed by the following layers : 
- `Flatten` layer
- `Dense` Layer : 10 neurons, softmax activation

> <div class="mark">Create the model</div><i class="fa fa-lightbulb-o "></i>

Documentation : 
- https://www.tensorflow.org/versions/r1.9/api_docs/python/tf/keras/applications/NASNetMobile
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

In [ ]:
NUM_CLASSES = 10
IMAGE_SIZE=32

conv_base = tf.keras.applications.NASNetMobile(weights='imagenet',
                                               include_top=False,
                                               input_shape=(32, 32, 3))

model = tf.keras.models.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

conv_base.trainable = False

In [ ]:
NUM_CLASSES = 10
IMAGE_SIZE=32

conv_base = # TODO

model = tf.keras.models.Sequential()
model.add(conv_base)
# TODO

In [ ]:
conv_base.trainable = False

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly classified).

You will implement the following compilation step for your Neural Network : 
- "adam" optimizer
- "categorical_crossentropy" loss
- metric : "accuracy"

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile

In [ ]:
optimizer = tf.optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Summarize the model

In [ ]:
model.summary()

## Train the model

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

You will fit the network with the following configurations :
- `x`: ds_train
- `epochs` : 5 (passes on the whole dataset)
- `steps_per_epoch`: 1000 steps
- `validation_data`: ds_test
- `validation_steps`: 10
- `callbacks`: tensorboard

You will also add a callback for launching TensorBoard to observe how the training is performing.

In [ ]:
LOG_DIR = './tensorboard/tf_keras_data_augment_transfer'

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1, update_freq="batch")

> <div class="mark">Fit the model with the above information.</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
shutil.rmtree(LOG_DIR, ignore_errors=True)

model.fit(x=ds_train,
         epochs=NUM_EPOCHS,
         steps_per_epoch=1000,
         validation_data=ds_test,
         validation_steps=10,
         callbacks=[tensorboard])

In [ ]:
shutil.rmtree(LOG_DIR, ignore_errors=True)

model. # TODO

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over the training data.

# Model Performance Evaluation

Now let's check that our model performs well on the test set too.

You can do this by calling the `evaluate` method of your network on the test set (use 300 for the `steps` argument).

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate

> <div class="mark">Evaluate the model performance on test set</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
model.evaluate(ds_test, steps=300)

In [ ]:
model. # TODO